# Student Loan Risk with Deep Learning

In [27]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [28]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [29]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

All data is either of type float or int. Therefore, no encoding necessary.

In [30]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

The two classes are balanced. No preprocessing required to balance the classes.

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [31]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
display(y[:5])

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [32]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=['credit_ranking'])

# Review the features DataFrame
display(X.head())

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [33]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [34]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Scale training and test datasets
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Display scaled training dataset
display(X_train_scaled)

array([[-0.56794375, -1.18088385, -0.19241296, ...,  0.73249574,
        -0.22553894, -0.31915735],
       [-0.00940983,  0.80065564, -0.04123135, ..., -0.03016065,
        -0.75844691, -1.15783431],
       [ 2.33643265, -0.84597577,  1.67216021, ..., -1.36480934,
         0.42579302,  0.05358797],
       ...,
       [-0.9589175 ,  1.07974571, -0.89792713, ...,  0.22405814,
        -0.9952949 ,  0.33314696],
       [-0.62379715,  0.54947458, -1.35147195, ...,  0.35116754,
        -0.46238693, -1.34420697],
       [ 0.43741731, -0.73433974,  1.16822152, ..., -0.66570765,
         0.18894504,  1.91731456]])

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [35]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
display(X.columns)
print(f"Number of features: {input_nodes}.")

Index(['payment_history', 'location_parameter', 'stem_degree_score',
       'gpa_ranking', 'alumni_success', 'study_major_code',
       'time_to_completion', 'finance_workshop_score', 'cohort_ranking',
       'total_loan_score', 'financial_aid_score'],
      dtype='object')

Number of features: 11.


Since there are 11 features, we will start with 27 neurons in each hidden layer which is about 2.5 times the number of features. We use the same nunber of neurons in both layers to start with.

Since we have two classes (0 and 1), we will use one node in the output layer.

In [42]:
# Define the number of hidden nodes for the first hidden layer
hidden_1_nodes = 6

# Define the number of hidden nodes for the second hidden layer
hidden_2_nodes = 3

# Define the number of neurons in the output layer
output_nodes = 1

In [43]:
# Create the Sequential model instance
nn_model = Sequential()

# Add the first hidden layer
nn_model.add(Dense(units=hidden_1_nodes, activation='relu', input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(Dense(units=hidden_2_nodes, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(Dense(units= output_nodes, activation='sigmoid'))

In [44]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 6)                 72        
                                                                 
 dense_7 (Dense)             (None, 3)                 21        
                                                                 
 dense_8 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [45]:
# Compile the Sequential model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
# Fit the model using 50 epochs and the training data
ft_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 2s 4ms/step - loss: 0.6951 - accuracy: 0.5638
Epoch 2/50
38/38 [==============================] - 0s 5ms/step - loss: 0.6628 - accuracy: 0.6781
Epoch 3/50
38/38 [==============================] - 0s 6ms/step - loss: 0.6407 - accuracy: 0.7098
Epoch 4/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6219 - accuracy: 0.7014
Epoch 5/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6055 - accuracy: 0.7089
Epoch 6/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5918 - accuracy: 0.7081
Epoch 7/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5786 - accuracy: 0.7164
Epoch 8/50
38/38 [==============================] - 0s 6ms/step - loss: 0.5677 - accuracy: 0.7298
Epoch 9/50
38/38 [==============================] - 0s 4ms/step - loss: 0.5576 - accuracy: 0.7348
Epoch 10/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5489 - accuracy: 0.7331
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [47]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss:.3f}, Accuracy: {model_accuracy:.3f}")

13/13 - 0s - loss: 0.5182 - accuracy: 0.7450 - 154ms/epoch - 12ms/step
Loss: 0.518, Accuracy: 0.745


Using 27 nodes in both hidden layers produced  
Loss: 0.543, Accuracy: 0.752  
Using 6 nodes in the first hidden layer and 3 nodes in the second hiiden layer produced  
Loss: 0.518, Accuracy: 0.745.

The difference is quite marginal. we will therefore continue to use the smaller number of units in the hidden layers.

### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [48]:
# Mount MyDrive
from google.colab import drive
drive.mount('/content/drive')

# Set the model's file path
file_path = Path('/content/drive/My Drive/neural-network-challenge-1/saved_model/student_loans.keras')

# Export your model to a keras file
nn_model.save(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [49]:
# Set the model's file path
file_path = Path('/content/drive/My Drive/neural-network-challenge-1/saved_model/student_loans.keras')

# Load the model to a new object
nn_model_reloaded = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [50]:
# Make predictions with the test data
predictions = nn_model_reloaded.predict(X_test_scaled, verbose=2)

# Display a sample of the predictions
display(predictions[:5])

13/13 - 0s - 89ms/epoch - 7ms/step


array([[0.34842265],
       [0.36822113],
       [0.80554765],
       [0.74414265],
       [0.97281885]], dtype=float32)

In [51]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(predictions, columns=['Predictions']).round(0)
display(predictions_df)


,Predictions
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [52]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df['Predictions']))

              precision    recall  f1-score   support

           0       0.71      0.77      0.74       188
           1       0.78      0.73      0.75       212

    accuracy                           0.74       400
   macro avg       0.75      0.75      0.74       400
weighted avg       0.75      0.74      0.75       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

**Answer:**  
The goal is to have as many students pay back their student loans as possible. It only makes sense to recommend a loan to a student if there is a very good chance, indeed, for them to pay it back on time.  
The ability for a given student to pay back their loan depends on what the final loan amount plus interest is versus (future) income and assets of the student. It also depends on the financial responsibility of the student.
Based on these factors, one would collect the following information:
*   *Loan amount*:  
    How much does the student want to borrow?  
    When will they be able to start paying back the loan? Will it be right away or only after they graduate and found a job?  
    What is the interest rate charged to the student?
*   *Income and assets*:  
    What is the student's current income (location adjusted)?  
    What are the student's assets?  
    Does the student have other loan payments (e.g., car loan)? If so how much?
    What is the student's major?  
    What is their GPA?  
    When will they (most likely) earn their degree?  
    What is their earnings potential, e.g., average yearly starting salary after they earn their degree (location adjusted)?  
    What is the average time it takes for a graduate with that major to find their first job after graduation?
*   *Financial Responsibility*:  
    What is the student's credit score?

In order to classify the student one would need to collect the following data for as many other students as possible:
*   Assets and income versus loan balance,
*   Other loan payments,
*   Major,
*   GPA,
*   Earning potential,
*   Credit score,
*   Did they repay their loan?.


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

**Answer:**  
I would use collaborative filtering using the data collected from other students to answer the question whether a particular student loan applicant would likely repay a student loan on time.  
There will not be enough data to use content-based filtering since it would only rely on the loan payment history of the loan applicant to assess the loan risk.  
I would not use context-based filtering because I am not sure what context to apply. Even if that was clear to me I would not know when to apply this context because loan repayment usually happens over years.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

**Answer:**
The biggest risks might be generated by the length of time it takes for loan repayments. Two risks associated with that are
1.   *Changing macro-economics*: This could lead to the student never reaching their earning potential and thereby being unable to pay back their loan. Since accurately forecasting economics many years into the future is impossible it is very difficult to mitigate against this risk.  
     It is hard to mitigate against this risk. One possible way to do that is to define a "Major Resiliency Score" assigning a larger weight to majors that lead to jobs that have shown in the past to fare better when economic indicators change.
2.   *Changing personal circumstances*: For example, someone could get sick, have an accident or die before the loan is repayed.  
     One way to mitigate against some of that risk would be to include the applicant's health history. However, that data is not available due to HIPAA (for good reason).



